In [1]:
import datetime as dt
import math

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
# from tooling.enums import AssetClass, Index, Spot, StrikeSpread
# from tooling.fetch import fetch_option_data, fetch_spot_data
# from tooling.filter import find_atm, option_tool

from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import (
    _fetch_batch,
    fetch_data,
    fetch_spot_data,
)

In [2]:
async def get_expiry(f_today):

    if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
        f_expiry = dt.date(2024, 1, 25)
    elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
        f_expiry = dt.date(2024, 1, 31)
    elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
        f_expiry = dt.date(2024, 2, 29)
    elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
        f_expiry = dt.date(2024, 2, 27)
    elif f_today < dt.date(2023, 9, 1):
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
    elif f_today >= dt.date(2023, 9, 1):
        if f_today.day < 24:
            days_to_wednesday = (2 - f_today.weekday()) % 7
            nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
            f_expiry = nearest_wednesday
            if nse.valid_days(
                start_date=nearest_wednesday, end_date=nearest_wednesday
            ).empty:
                f_expiry = nearest_wednesday - dt.timedelta(days=1)
        else:
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(
                start_date=nearest_thursday, end_date=nearest_thursday
            ).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_expiry_finnifty(f_today):

    days_to_thursday = (1 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_expiry_nifty(f_today):

    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_expiry_midcpnifty(f_today):

    days_to_thursday = (0 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


import datetime as dt


async def get_monthly_expiry_nifty(input_date):
    # Get the last day of the current month
    current_month_last_day = (
        input_date.replace(day=28) + dt.timedelta(days=4)
    ).replace(day=1) - dt.timedelta(days=1)

    # Find the last Thursday of the current month
    last_thursday_current_month = current_month_last_day - dt.timedelta(
        days=(current_month_last_day.weekday() - 3) % 7
    )

    # Check if the current date is less than the last Thursday of the current month
    if input_date < last_thursday_current_month:
        last_thursday = last_thursday_current_month
    else:
        # If the current date has passed the last Thursday, find the last Thursday of the next month
        next_month = (input_date.month % 12) + 1
        next_month_year = input_date.year if next_month > 1 else input_date.year + 1

        # Get the last day of the next month (considering February correctly)
        if next_month == 2:  # February
            if next_month_year % 4 == 0 and (
                next_month_year % 100 != 0 or next_month_year % 400 == 0
            ):
                last_day_of_next_month = 29  # Leap year
            else:
                last_day_of_next_month = 28  # Non-leap year
        else:
            # Calculate the last day of the next month
            last_day_of_next_month = (
                dt.date(next_month_year, next_month, 1) + dt.timedelta(days=31)
            ).replace(day=1) - dt.timedelta(days=1)
            last_day_of_next_month = (
                last_day_of_next_month.day
            )  # Extract the day as an integer

        # Create a date for the last day of the next month
        last_day_of_next_month_date = dt.date(
            next_month_year, next_month, last_day_of_next_month
        )

        # Find the last Thursday of the next month
        last_thursday = last_day_of_next_month_date - dt.timedelta(
            days=(last_day_of_next_month_date.weekday() - 3) % 7
        )

    # Validate if the last Thursday is a trading day
    if nse.valid_days(start_date=last_thursday, end_date=last_thursday).empty:
        # If it's a holiday, find the previous valid trading day
        last_thursday -= dt.timedelta(days=1)
        while nse.valid_days(start_date=last_thursday, end_date=last_thursday).empty:
            last_thursday -= dt.timedelta(days=1)

    return last_thursday


async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [3]:
# bnf_pandas = pd.read_csv("../data/bnf_min.csv")
bnf_pandas = pd.read_csv("../data/nifty_min.csv")
# bnf_pandas = pd.read_csv("../data/fin_min.csv")
# bnf_pandas = pd.read_csv("../data/midcp_min.csv")
# bnf_pandas = pd.read_csv("../data/sensex_min.csv")
# bnf_pandas = pd.read_csv("../data/bankex_min.csv")

In [4]:
bnf_pandas.columns = ['index', 'datetime', 'o', 'h', 'l', 'c', 'v']
bnf_pandas.tail()

,index,datetime,o,h,l,c,v
743635,nifty,2025-02-28 15:25:00,22110.4500,22119.0500,22109.2500,22118.8000,0
743636,nifty,2025-02-28 15:26:00,22118.6000,22123.8500,22113.2000,22114.7000,0
743637,nifty,2025-02-28 15:27:00,22114.0000,22120.1500,22108.4500,22114.1500,0
743638,nifty,2025-02-28 15:28:00,22114.1500,22121.2500,22106.1000,22113.8500,0
743639,nifty,2025-02-28 15:29:00,22112.5500,22127.4000,22109.3500,22127.4000,0


In [5]:
# If Stocks Data ...
bnf_pandas["datetime"] = pd.to_datetime(bnf_pandas["datetime"])
bnf_pandas["datetime"] = bnf_pandas["datetime"].dt.tz_localize(None)
bnf_pandas = bnf_pandas[bnf_pandas["datetime"].dt.year >= 2017]
# bnf_pandas.drop(columns=["time"], inplace=True)
# bnf_pandas

In [6]:
bnf = pl.DataFrame(bnf_pandas)
print(type(bnf))
# bnf

<class 'polars.dataframe.frame.DataFrame'>


In [7]:
bnf = bnf.with_columns([pl.col("datetime").alias("index")]).drop("datetime")
bnf = bnf.with_columns(pl.col("index").alias("datetime"))

In [8]:
# bnf = bnf.rename({"open": "o", "high": "h", "low": "l", "close": "c", "volume": "v"})

In [9]:
def resample(data, timeframe, offset=None):
    agg_list = [
        pl.col("o").first().alias("o"),
        pl.col("h").max().alias("h"),
        pl.col("l").min().alias("l"),
        pl.col("c").last().alias("c"),
    ]
    if timeframe == '10m':
        offset = '5m'
    if "v" in data.columns:
        agg_list.append(pl.col("v").sum().alias("v"))
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(agg_list)
    )


In [10]:
# def generate_signals(df, n=5):
#     import numpy as np
#     import pandas as pd

#     # Ensure required columns are present
#     required_cols = {'o', 'h', 'l', 'c', 'datetime'}
#     if not required_cols.issubset(df.columns):
#         raise ValueError(f"DataFrame must contain columns: {required_cols}")
    
#     # Ensure datetime is in datetime format
#     if not np.issubdtype(df['datetime'].dtype, np.datetime64):
#         df['datetime'] = pd.to_datetime(df['datetime'])
    
#     # Calculate the low of the previous n candles
#     df['Prev_N_Low'] = df['l'].rolling(window=n).min().shift(1)
    
#     # Generate sell signal
#     df['Sell Signal'] = df['c'] < df['Prev_N_Low']
    
#     # Drop intermediate columns if not needed
#     df.drop(columns=['Prev_N_Low'], inplace=True, errors='ignore')
    
#     return df


In [11]:
# # USING SMA LOW for ENTRY
# def generate_signals(df, st_num=3, ema=5, pct=0.9):
#     """
#     Calculate signals for reversal selling strategy with SMA-based condition.
    
#     Parameters:
#     df (pd.DataFrame): Input DataFrame with columns 'datetime', 'o', 'h', 'l', 'c', and optionally 'v'.
#     st_num (int): Period for short-term SMA and low calculation.
#     ema (int): Period for EMA calculation.
#     pct (float): Percentage threshold for high price comparison.
    
#     Returns:
#     pd.DataFrame: DataFrame with additional columns 'Reversal Sell Signal' and tracking indicators.
#     """
#     # Ensure the DataFrame has the required columns
#     required_columns = {'datetime', 'h', 'l', 'c'}
#     if not required_columns.issubset(df.columns):
#         raise ValueError(f"Input DataFrame must contain columns: {required_columns}")
    
#     # Calculate short-term SMA of the lows
#     df['SMA_Low'] = df['l'].rolling(window=st_num).mean().shift(1)
    
#     # Calculate a very short-term EMA (e.g., 5-period)
#     df['EMA_5'] = df['c'].ewm(span=ema, adjust=False).mean()
    
#     # Calculate daily high till now
#     df['daily_high_till_now'] = df.groupby(df['datetime'].dt.date)['h'].cummax()
    
#     # Define Reversal Sell Signal
#     df['Sell Signal'] = (
#         (df['h'] > (pct * df['daily_high_till_now'].shift(1))) &  # Price is near or above the daily high
#         (df['c'] < df['SMA_Low'])  # Close below SMA of the lows
#     ).astype(int)
    
#     return df


In [12]:
def generate_signals(df, st_num=3, ema=5, pct=0.9):
    """
    Calculate signals for reversal selling strategy.
    
    Parameters:
    df (pd.DataFrame): Input DataFrame with columns 'datetime', 'o', 'h', 'l', 'c', and optionally 'v'.
    
    Returns:
    pd.DataFrame: DataFrame with additional columns 'Reversal Sell Signal' and tracking indicators.
    """
    # Ensure the DataFrame has the required columns
    required_columns = {'datetime', 'h', 'l', 'c'}
    if not required_columns.issubset(df.columns):
        raise ValueError(f"Input DataFrame must contain columns: {required_columns}")
    
    # Calculate a 3-period high (short-term high)
    df['short_term_low'] = df['l'].rolling(window=st_num).min().shift(1)

    # Calculate a very short-term EMA (e.g., 5-period)
    df['EMA_5'] = df['c'].ewm(span=ema, adjust=False).mean()
    
    # Calculate daily high till now
    df['daily_high_till_now'] = df.groupby(df['datetime'].dt.date)['h'].cummax()
    
    # Define Reversal Sell Signal
    # df['Sell Signal'] = (
    #     (df['h'] > (pct*df['daily_high_till_now'].shift(1))) &  # Price is near or above the daily high
    #     (df['c'] < df['short_term_low']) &  # Closing below the short-term high (early weakness)
    #     (df['c'] < df['EMA_5'])  # Close below very short-term EMA for confirmation
    # ).astype(int)
    df['Sell Signal'] = (
        (df['h'] > (pct*df['daily_high_till_now'].shift(1))) &  # Price is near or above the daily high
        (df['c'] < df['short_term_low'])).astype(int)
    
    return df


In [13]:
data = bnf_pandas.copy()
# data['datetime'] = pd.to_datetime(data['datetime'].dt.date)
print(data["datetime"].tail())
trading_days_set = set(data["datetime"].dt.date)
# sorted(trading_days_set)

743635   2025-02-28 15:25:00
743636   2025-02-28 15:26:00
743637   2025-02-28 15:27:00
743638   2025-02-28 15:28:00
743639   2025-02-28 15:29:00
Name: datetime, dtype: datetime64[ns]


In [14]:
# GLOBAL VARIABLES

INSTRUMENT = "NIFTY"
INDEX = "nifty"
INDEX_MROUND = 50
# INDEX_MROUND=100

# INSTRUMENT = "NIFTY"
# INDEX = "nifty"
# INDEX_MROUND=50

PORTFOLIO_VALUE = 10_00_000
INDEX_LEV = 8
RPT_CE = 0.03
RPT_PE = 0.03
SLIPPAGE = 0.01
TF = "10m"

# SIGNAL_MA = 20
# NUM_OF_CANDELS = 1
# T_MA=20
# TARGET=100

In [15]:
bnf = resample(bnf, TF)
data = bnf.to_pandas()
data.tail()
# data[['MA','signal_spot']]=MA(data,200)
# data[data['signal_spot']==1].head(50)
# bnf
# data.tail(50)
# data[data['datetime'].dt.date == dt.date(2024, 4, 29)]

,datetime,o,h,l,c,v
75391,2025-02-28 14:45:00,22172.3000,22198.7500,22157.3000,22172.8500,0
75392,2025-02-28 14:55:00,22170.1500,22176.5500,22127.0500,22137.6000,0
75393,2025-02-28 15:05:00,22137.2500,22138.3000,22117.2500,22124.6500,0
75394,2025-02-28 15:15:00,22124.8500,22127.6500,22104.8500,22110.4500,0
75395,2025-02-28 15:25:00,22110.4500,22127.4000,22106.1000,22127.4000,0


In [16]:
async def ce_trade(data, st_high, ema, pct):
    df = data.copy()

    start_date = dt.date(2019, 1, 1)
    end_date = dt.date(2025, 2, 28)

    current_date = start_date

    combined_trades = pd.DataFrame()
    total_trades = pd.DataFrame()
    time_of_day = dt.time(9, 15)
    trade_book = []
    ce_lowest_low = float("inf")
    ce_highest_high = float("-inf")
    entry_rsi = 0

    while current_date < end_date:
        print(current_date)
        entry = 0
        initial_sl = 0
        exit = 0
        in_ce_trade = False
        in_pe_trade = False
        # signal_exist=False

        points_captured = 0
        remark = ""
        trailing_active = False
        tsl = 0
        stop_trading = False
        is_gap_ce_sl = False
        previous_ce_sl_hit = False
        current_date_increament_flag = False
        # tsl_high = 0

        starting_time = dt.time(9, 15)

        ending_time = dt.time(15, 30)

        if not in_ce_trade and current_date in trading_days_set:

            ce_search_datetime = dt.datetime.combine(current_date, time_of_day)
            # print(f'current date : {ce_search_datetime}')

            spot_open = df.loc[df["datetime"] >= ce_search_datetime, "o"].iloc[0]
            # print(f'spot open : {spot_open}')
            # spot_atm = int(round(spot_open / INDEX_MROUND) * INDEX_MROUND)
            spot_atm = int(
                math.ceil(spot_open / INDEX_MROUND) * INDEX_MROUND
            )  ##ROUNDS TO NEAREST 500 OTM
            # print(f'spot atm : {spot_atm}')
            # nearest_expiry = await get_expiry(current_date)
            nearest_expiry = await get_expiry_nifty(current_date)
            # if current_date== nearest_expiry:
            #     next_expiry_passing_value = current_date + dt.timedelta(days=1)
            #     nearest_expiry = await get_expiry_nifty( next_expiry_passing_value)
            # print(f'passing date for expry : {current_date}')
            # nearest_expiry = await get_monthly_expiry_nifty(current_date)
            # print(f'nearest expiry{nearest_expiry}')
            selected_strike_ce = spot_atm
            # print(f'selected strike CE : {selected_strike_ce}')
            ce_df = await fetch_data(
                index=INDEX,
                start_date=nearest_expiry - dt.timedelta(days=7),
                start_time=starting_time,
                end_date=nearest_expiry,
                end_time=ending_time,
                strike=selected_strike_ce,
                asset_class="C",
                expiry=nearest_expiry,
            )
            # print(ce_df)
            if ce_df is not None and not isinstance(ce_df, str):
                # print('new data fetched CE')
                data_ce = True
                ce_df = ce_df.select(["datetime", "o", "h", "l", "c", "v"])
                ce_df = resample(ce_df, TF)
                ce_df_pandas = ce_df.to_pandas()
                ce_df = generate_signals(ce_df_pandas, st_high, ema, pct)
                # ce_df = calculate_signals(ce_df_pandas)
                # print(ce_df.to_string())
            else:
                data_ce = False
                current_date += dt.timedelta(days=1)
                continue

            if data_ce:

                for i in range(0, len(ce_df)):
                    current_candle = ce_df.iloc[i]
                    current_candle_open = ce_df.iloc[i]["o"]
                    current_candle_high = ce_df.iloc[i]["h"]
                    current_candle_low = ce_df.iloc[i]["l"]
                    current_candle_close = ce_df.iloc[i]["c"]

                    previous_candle_low = ce_df.iloc[i - 1]["l"]
                    previous_candle_close = ce_df.iloc[i - 1]["c"]
                    

                    expiry = nearest_expiry
                    strike = selected_strike_ce
                    asset_class = "C"
                    # print(ce_df.iloc[i])

                    signal = ce_df.iloc[i - 1]["Sell Signal"]
                    # candle_condition = (ce_df.iloc[i-1]['daily_high_till_now'] - previous_candle_close) > 10

                    if ce_df.iloc[i]["datetime"] >= ce_search_datetime:


                        if (
                            not previous_ce_sl_hit
                            and not in_ce_trade
                            and signal
                            # and current_candle_low < previous_candle_low
                            and ce_df.iloc[i]["datetime"].time() > time_of_day
                            and (
                                (nearest_expiry - ce_df.iloc[i]["datetime"].date()).days
                                >= 0
                                and (
                                    nearest_expiry - ce_df.iloc[i]["datetime"].date()
                                ).days
                                < 8
                            )
                            and ce_df.iloc[i]["datetime"].time() < dt.time(15, 25)
                            # and candle_condition
                        ):
                            # print(ce_df.iloc[i-1])
                            # print(f'entry found {previous_candle_low}')
                            # print(f'entry datetime {ce_df.iloc[i]["datetime"]}')
                            
                            # today_data = ce_df[ce_df['datetime'].dt.date == current_candle['datetime'].date()]
                            # day_high = today_data.iloc[0 : i-1]['h'].max()
                            # print(today_data.to_string())
                            
                            entry = current_candle_open
                            entry_date = ce_df.iloc[i-1]["datetime"].date()
                            entry_time = ce_df.iloc[i-1]["datetime"].time()
                            # initial_sl = ce_df.iloc[i - SL_CANDLES_NUM : i]["h"].max()
                            # initial_sl = day_high
                            initial_sl = ce_df.iloc[i-1]['daily_high_till_now']
                            in_ce_trade = True
                            ce_lowest_low = float("inf")
                            ce_highest_high = float("-inf")
                            # print(f'initial SL : {initial_sl}')
                            # entry_rsi = ce_df.iloc[i-1]['RSI']

                            qty = RPT_CE * PORTFOLIO_VALUE / (initial_sl - entry)
                            if (
                                (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                            ) * 100 > 250:
                                qty = PORTFOLIO_VALUE * INDEX_LEV / strike * 2.5

                            # print(f'qty : {qty}')

                        # While in trade, track the highest high and lowest low
                        if in_ce_trade:
                            # Track the highest high
                            ce_highest_high = max(ce_highest_high, current_candle_high)

                            # Track the lowest low
                            ce_lowest_low = min(ce_lowest_low, current_candle_low)

                            # if all(
                            #     ce_df.loc[i - j, "h"] <= ce_df.loc[i - fractal_num, "h"]
                            #     for j in range(0, ((fractal_num * 2) + 1))
                            # ):
                            #     tsl_high = ce_df.loc[i - fractal_num, "h"]

                        if (
                            in_ce_trade
                            and ce_df.iloc[i]["datetime"].time() == dt.time(9, 15)
                            and current_candle_open > initial_sl
                        ):

                            # print(ce_df.iloc[i])
                            # print(f'GAP sl hit {initial_sl}')
                            # print(f'GAP sl datetime {ce_df.iloc[i]["datetime"]}')
                            exit = current_candle_close
                            in_ce_trade = False
                            stop_trading = False
                            previous_ce_sl_hit = True
                            is_gap_ce_sl = False
                            points_captured = entry - exit
                            exit_time = ce_df.iloc[i]["datetime"].time()
                            slippage = SLIPPAGE * (entry + exit)
                            pnl = qty * (points_captured - slippage)
                            remark = "Gap SL hit"
                            weekday_int = entry_date.weekday()
                            weekday_name = [
                                "Monday",
                                "Tuesday",
                                "Wednesday",
                                "Thursday",
                                "Friday",
                                "Saturday",
                                "Sunday",
                            ][weekday_int]
                            trade = {
                                "date": entry_date,
                                "day": weekday_name,
                                "expiry": expiry,
                                "DTE": (nearest_expiry - entry_date).days,
                                # 'atm' : atm,
                                # 'scrip' : index ,
                                "strike": strike,
                                "type": asset_class,
                                "Entry Price": entry,
                                "Entry Time": entry_time,
                                "initial sl": initial_sl,
                                # "TSL": tsl_high,
                                # 'OTM Entry' : otm_entry,
                                "Exit Price": exit,
                                "Exit date": ce_df.iloc[i]["datetime"].date(),
                                "Exit Time": exit_time,
                                'RSI on Entry': entry_rsi,
                                # 'OTM EXIT ' : otm_exit,
                                "Remark": remark,
                                "Points Captured": points_captured,
                                "Slippage": slippage,
                                # 'OTM cost' : otm_exit-otm_entry,
                                "Qty": qty,
                                "PnL": pnl,
                                "ROI%": (pnl / PORTFOLIO_VALUE) * 100,
                                "Trade Year": ce_df.iloc[i]["datetime"].year,
                                "Trade Month": ce_df.iloc[i]["datetime"].month,
                                "Highest High": ce_highest_high,  # Add highest high to trade data
                                "Lowest Low": ce_lowest_low,  # Add lowest low to trade data
                                "Max ROI%": (
                                    (qty * (entry - ce_lowest_low)) / PORTFOLIO_VALUE
                                )
                                * 100,
                                "Margin": (
                                    (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                                )
                                * 100,
                            }
                            # print(trade)
                            trade_book.append(trade)
                            # tsl_high = 0
                            points_captured = 0
                            current_date = ce_df.iloc[i]["datetime"].date()
                            current_date_increament_flag = True
                            time_of_day = ce_df.iloc[i]["datetime"].time()
                            # print(f'current date changed to : {current_date} and time to {time_of_day}')
                            break

                        if in_ce_trade and current_candle_high > initial_sl:
                            # print(ce_df.iloc[i])
                            # print(f'initial sl hit {initial_sl}')
                            # print(f'initial sl datetime {ce_df.iloc[i]["datetime"]}')
                            exit = initial_sl
                            otm_datetime = ce_df.iloc[i]["datetime"]
                            in_ce_trade = False
                            stop_trading = False
                            previous_ce_sl_hit = True
                            is_gap_ce_sl = False
                            points_captured = entry - exit
                            exit_time = ce_df.iloc[i]["datetime"].time()
                            slippage = SLIPPAGE * (entry + exit)
                            pnl = qty * (points_captured - slippage)
                            # pnl=(qty*(points_captured-slippage))-qty*(otm_exit-otm_entry)
                            remark = "SL hit"
                            weekday_int = entry_date.weekday()
                            weekday_name = [
                                "Monday",
                                "Tuesday",
                                "Wednesday",
                                "Thursday",
                                "Friday",
                                "Saturday",
                                "Sunday",
                            ][weekday_int]
                            trade = {
                                "date": entry_date,
                                "day": weekday_name,
                                "expiry": expiry,
                                "DTE": (nearest_expiry - entry_date).days,
                                # 'atm' : atm,
                                # 'scrip' : index ,
                                "strike": strike,
                                "type": asset_class,
                                "Entry Price": entry,
                                "Entry Time": entry_time,
                                "initial sl": initial_sl,
                                # "TSL": tsl_high,
                                # 'OTM Entry' : otm_entry,
                                "Exit Price": exit,
                                "Exit date": ce_df.iloc[i]["datetime"].date(),
                                "Exit Time": exit_time,
                                'RSI on Entry': entry_rsi,
                                # 'OTM EXIT ' : otm_exit,
                                "Remark": remark,
                                "Points Captured": points_captured,
                                "Slippage": slippage,
                                # 'OTM cost' : otm_exit-otm_entry,
                                "Qty": qty,
                                "PnL": pnl,
                                "ROI%": (pnl / PORTFOLIO_VALUE) * 100,
                                "Trade Year": ce_df.iloc[i]["datetime"].year,
                                "Trade Month": ce_df.iloc[i]["datetime"].month,
                                "Highest High": ce_highest_high,  # Add highest high to trade data
                                "Lowest Low": ce_lowest_low,  # Add lowest low to trade data
                                "Max ROI%": (
                                    (qty * (entry - ce_lowest_low)) / PORTFOLIO_VALUE
                                )
                                * 100,
                                "Margin": (
                                    (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                                )
                                * 100,
                            }
                            # print(trade)
                            trade_book.append(trade)
                            # tsl_high = 0
                            points_captured = 0
                            current_date = ce_df.iloc[i]["datetime"].date()
                            current_date_increament_flag = True
                            time_of_day = ce_df.iloc[i]["datetime"].time()
                            # print(f'current date changed to : {current_date} and time to {time_of_day}')
                            break

                        if (
                            in_ce_trade
                            and ce_df.iloc[i]["datetime"].date() == nearest_expiry
                            and ce_df.iloc[i]['datetime'].time() >= dt.time(15, 20)
                        ):
                            # print(ce_df.iloc[i])
                            # print(f'EOD exit {current_candle_close}')
                            # print(f'EOD datetime {ce_df.iloc[i]["datetime"]}')
                            exit = current_candle_close
                            otm_datetime = ce_df.iloc[i]["datetime"]
                            in_ce_trade = False
                            previous_ce_sl_hit = True
                            is_gap_ce_sl = False
                            points_captured = entry - exit
                            exit_time = ce_df.iloc[i]["datetime"].time()
                            slippage = SLIPPAGE * (entry + exit)
                            pnl = qty * (points_captured - slippage)
                            # pnl=(qty*(points_captured-slippage))-qty*(otm_exit-otm_entry)
                            remark = "EOD exit"
                            weekday_int = entry_date.weekday()
                            weekday_name = [
                                "Monday",
                                "Tuesday",
                                "Wednesday",
                                "Thursday",
                                "Friday",
                                "Saturday",
                                "Sunday",
                            ][weekday_int]
                            trade = {
                                "date": entry_date,
                                "day": weekday_name,
                                "expiry": expiry,
                                "DTE": (nearest_expiry - entry_date).days,
                                # 'atm' : atm,
                                # 'scrip' : index ,
                                "strike": strike,
                                "type": asset_class,
                                "Entry Price": entry,
                                "Entry Time": entry_time,
                                "initial sl": initial_sl,
                                # "TSL": tsl_high,
                                # 'OTM Entry' : otm_entry,
                                "Exit Price": exit,
                                "Exit date": ce_df.iloc[i]["datetime"].date(),
                                "Exit Time": exit_time,
                                'RSI on Entry': entry_rsi,
                                # 'OTM EXIT ' : otm_exit,
                                "Remark": remark,
                                "Points Captured": points_captured,
                                "Slippage": slippage,
                                # 'OTM cost' : otm_exit-otm_entry,
                                "Qty": qty,
                                "PnL": pnl,
                                "ROI%": (pnl / PORTFOLIO_VALUE) * 100,
                                "Trade Year": ce_df.iloc[i]["datetime"].year,
                                "Trade Month": ce_df.iloc[i]["datetime"].month,
                                "Highest High": ce_highest_high,  # Add highest high to trade data
                                "Lowest Low": ce_lowest_low,  # Add lowest low to trade data
                                "Max ROI%": (
                                    (qty * (entry - ce_lowest_low)) / PORTFOLIO_VALUE
                                )
                                * 100,
                                "Margin": (
                                    (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                                )
                                * 100,
                            }
                            # print(trade)
                            trade_book.append(trade)
                            # tsl_high = 0
                            points_captured = 0
                            current_date = nearest_expiry + dt.timedelta(days=1)
                            current_date_increament_flag = True
                            time_of_day = dt.time(9, 15)
                            # print(f'current date increased by 1 on expiry : {current_date}')
                            break

                        if (
                            not in_ce_trade
                            and not previous_ce_sl_hit
                            and ce_df.iloc[i]["datetime"].time() > dt.time(15, 15)
                        ):
                            # print('inside exoiry non trade date increment')
                            current_date = current_date + dt.timedelta(days=1)
                            current_date_increament_flag = True
                            time_of_day = dt.time(9, 15)
                            break

        if not current_date_increament_flag:
            current_date = current_date + dt.timedelta(days=1)
            current_date_increament_flag = False

    trade_book_df = pd.DataFrame(trade_book)

    return trade_book_df

In [17]:
async def pe_trade(data, st_high, ema, pct):
    df = data.copy()

    start_date = dt.date(2019, 1, 1)
    end_date = dt.date(2025, 2, 28)
    
    current_date = start_date

    combined_trades = pd.DataFrame()
    total_trades = pd.DataFrame()
    time_of_day = dt.time(9, 15)
    trade_book = []
    pe_lowest_low = float("inf")
    pe_highest_high = float("-inf")
    entry_rsi = 0

    while current_date < end_date:
        print(current_date)
        entry = 0
        initial_sl = 0
        exit = 0
        in_pe_trade = False
        in_pe_trade = False
        # signal_exist=False

        points_captured = 0
        remark = ""
        trailing_active = False
        tsl = 0
        stop_trading = False
        is_gap_pe_sl = False
        previous_pe_sl_hit = False
        current_date_increament_flag = False
        # tsl_high = 0

        starting_time = dt.time(9, 15)

        ending_time = dt.time(15, 30)

        if not in_pe_trade and current_date in trading_days_set:

            pe_search_datetime = dt.datetime.combine(current_date, time_of_day)
            # print(f'current date : {pe_search_datetime}')

            spot_open = df.loc[df["datetime"] >= pe_search_datetime, "o"].iloc[0]
            # print(f'spot open : {spot_open}')
            # spot_atm = int(round(spot_open / INDEX_MROUND) * INDEX_MROUND)
            spot_atm = int(
                math.floor(spot_open / INDEX_MROUND) * INDEX_MROUND
            )  ##ROUNDS TO NEAREST 500 OTM
            # print(f'spot atm : {spot_atm}')
            # nearest_expiry = await get_expiry(current_date)
            nearest_expiry = await get_expiry_nifty(current_date)
            # if current_date== nearest_expiry:
            #     next_expiry_passing_value = current_date + dt.timedelta(days=1)
            #     nearest_expiry = await get_expiry_nifty( next_expiry_passing_value)
            # print(f'passing date for expry : {current_date}')
            # nearest_expiry = await get_monthly_expiry_nifty(current_date)
            # print(f'nearest expiry{nearest_expiry}')
            selected_strike_pe = spot_atm
            # print(f'selected strike PE : {selected_strike_pe}')
            pe_df = await fetch_data(
                index=INDEX,
                start_date=nearest_expiry - dt.timedelta(days=7),
                start_time=starting_time,
                end_date=nearest_expiry,
                end_time=ending_time,
                strike=selected_strike_pe,
                asset_class="P",
                expiry=nearest_expiry,
            )
            if pe_df is not None and not isinstance(pe_df, str):
                # print('new data fetched PE')
                data_pe = True
                pe_df = pe_df.select(["datetime", "o", "h", "l", "c", "v"])
                pe_df = resample(pe_df, TF)
                pe_df_pandas = pe_df.to_pandas()
                pe_df = generate_signals(pe_df_pandas, st_high, ema, pct)
                # pe_df = calculate_signals(pe_df_pandas)
                # print(pe_df.to_string())
            else:
                data_pe = False
                current_date += dt.timedelta(days=1)
                continue

            if data_pe:

                for i in range(0, len(pe_df)):
                    current_candle = pe_df.iloc[i]
                    current_candle_open = pe_df.iloc[i]["o"]
                    current_candle_high = pe_df.iloc[i]["h"]
                    current_candle_low = pe_df.iloc[i]["l"]
                    current_candle_close = pe_df.iloc[i]["c"]

                    previous_candle_low = pe_df.iloc[i - 1]["l"]
                    previous_candle_close = pe_df.iloc[i - 1]["c"]
                    

                    expiry = nearest_expiry
                    strike = selected_strike_pe
                    asset_class = "P"
                    # print(pe_df.iloc[i])

                    signal = pe_df.iloc[i - 1]["Sell Signal"]
                    # candle_condition = (pe_df.iloc[i-1]['daily_high_till_now'] - previous_candle_close) > 10

                    if pe_df.iloc[i]["datetime"] >= pe_search_datetime:


                        if (
                            not previous_pe_sl_hit
                            and not in_pe_trade
                            and signal
                            # and current_candle_low < previous_candle_low
                            and pe_df.iloc[i]["datetime"].time() > time_of_day
                            and (
                                (nearest_expiry - pe_df.iloc[i]["datetime"].date()).days
                                >= 0
                                and (
                                    nearest_expiry - pe_df.iloc[i]["datetime"].date()
                                ).days
                                < 8
                            )
                            and pe_df.iloc[i]["datetime"].time() < dt.time(15, 25)
                            # and candle_condition
                        ):
                            # print(pe_df.iloc[i-1])
                            # print(f'entry found {previous_candle_low}')
                            # print(f'entry datetime {pe_df.iloc[i]["datetime"]}')
                            
                            # today_data = pe_df[pe_df['datetime'].dt.date == current_candle['datetime'].date()]
                            # day_high = today_data.iloc[0 : i]['h'].max()
                            # print(today_data.to_string())
                            
                            entry = current_candle_open
                            entry_date = pe_df.iloc[i-1]["datetime"].date()
                            entry_time = pe_df.iloc[i-1]["datetime"].time()
                            # initial_sl = pe_df.iloc[i - SL_CANDLES_NUM : i]["h"].max()
                            # initial_sl = day_high
                            initial_sl = pe_df.iloc[i-1]['daily_high_till_now']
                            in_pe_trade = True
                            pe_lowest_low = float("inf")
                            pe_highest_high = float("-inf")
                            # entry_rsi = pe_df.iloc[i-1]['RSI']

                            qty = RPT_PE * PORTFOLIO_VALUE / (initial_sl - entry)
                            if (
                                (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                            ) * 100 > 250:
                                qty = PORTFOLIO_VALUE * INDEX_LEV / strike * 2.5

                        # While in trade, track the highest high and lowest low
                        if in_pe_trade:
                            # Track the highest high
                            pe_highest_high = max(pe_highest_high, current_candle_high)

                            # Track the lowest low
                            pe_lowest_low = min(pe_lowest_low, current_candle_low)

                            # if all(
                            #     pe_df.loc[i - j, "h"] <= pe_df.loc[i - fractal_num, "h"]
                            #     for j in range(0, ((fractal_num * 2) + 1))
                            # ):
                            #     tsl_high = pe_df.loc[i - fractal_num, "h"]

                        if (
                            in_pe_trade
                            and pe_df.iloc[i]["datetime"].time() == dt.time(9, 15)
                            and current_candle_open > initial_sl
                        ):

                            # print(pe_df.iloc[i])
                            # print(f'GAP sl hit {initial_sl}')
                            # print(f'GAP sl datetime {pe_df.iloc[i]["datetime"]}')
                            exit = current_candle_close
                            in_pe_trade = False
                            stop_trading = False
                            previous_pe_sl_hit = True
                            is_gap_pe_sl = False
                            points_captured = entry - exit
                            exit_time = pe_df.iloc[i]["datetime"].time()
                            slippage = SLIPPAGE * (entry + exit)
                            pnl = qty * (points_captured - slippage)
                            remark = "Gap SL hit"
                            weekday_int = entry_date.weekday()
                            weekday_name = [
                                "Monday",
                                "Tuesday",
                                "Wednesday",
                                "Thursday",
                                "Friday",
                                "Saturday",
                                "Sunday",
                            ][weekday_int]
                            trade = {
                                "date": entry_date,
                                "day": weekday_name,
                                "expiry": expiry,
                                "DTE": (nearest_expiry - entry_date).days,
                                # 'atm' : atm,
                                # 'scrip' : index ,
                                "strike": strike,
                                "type": asset_class,
                                "Entry Price": entry,
                                "Entry Time": entry_time,
                                "initial sl": initial_sl,
                                # "TSL": tsl_high,
                                # 'OTM Entry' : otm_entry,
                                "Exit Price": exit,
                                "Exit date": pe_df.iloc[i]["datetime"].date(),
                                "Exit Time": exit_time,
                                'RSI on Entry': entry_rsi,
                                # 'OTM EXIT ' : otm_exit,
                                "Remark": remark,
                                "Points Captured": points_captured,
                                "Slippage": slippage,
                                # 'OTM cost' : otm_exit-otm_entry,
                                "Qty": qty,
                                "PnL": pnl,
                                "ROI%": (pnl / PORTFOLIO_VALUE) * 100,
                                "Trade Year": pe_df.iloc[i]["datetime"].year,
                                "Trade Month": pe_df.iloc[i]["datetime"].month,
                                "Highest High": pe_highest_high,  # Add highest high to trade data
                                "Lowest Low": pe_lowest_low,  # Add lowest low to trade data
                                "Max ROI%": (
                                    (qty * (entry - pe_lowest_low)) / PORTFOLIO_VALUE
                                )
                                * 100,
                                "Margin": (
                                    (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                                )
                                * 100,
                            }
                            # print(trade)
                            trade_book.append(trade)
                            # tsl_high = 0
                            points_captured = 0
                            current_date = pe_df.iloc[i]["datetime"].date()
                            current_date_increament_flag = True
                            time_of_day = pe_df.iloc[i]["datetime"].time()
                            # print(f'current date changed to : {current_date} and time to {time_of_day}')
                            break

                        if in_pe_trade and current_candle_high > initial_sl:
                            # print(pe_df.iloc[i])
                            # print(f'initial sl hit {initial_sl}')
                            # print(f'initial sl datetime {pe_df.iloc[i]["datetime"]}')
                            exit = initial_sl
                            otm_datetime = pe_df.iloc[i]["datetime"]
                            in_pe_trade = False
                            stop_trading = False
                            previous_pe_sl_hit = True
                            is_gap_pe_sl = False
                            points_captured = entry - exit
                            exit_time = pe_df.iloc[i]["datetime"].time()
                            slippage = SLIPPAGE * (entry + exit)
                            pnl = qty * (points_captured - slippage)
                            # pnl=(qty*(points_captured-slippage))-qty*(otm_exit-otm_entry)
                            remark = "SL hit"
                            weekday_int = entry_date.weekday()
                            weekday_name = [
                                "Monday",
                                "Tuesday",
                                "Wednesday",
                                "Thursday",
                                "Friday",
                                "Saturday",
                                "Sunday",
                            ][weekday_int]
                            trade = {
                                "date": entry_date,
                                "day": weekday_name,
                                "expiry": expiry,
                                "DTE": (nearest_expiry - entry_date).days,
                                # 'atm' : atm,
                                # 'scrip' : index ,
                                "strike": strike,
                                "type": asset_class,
                                "Entry Price": entry,
                                "Entry Time": entry_time,
                                "initial sl": initial_sl,
                                # "TSL": tsl_high,
                                # 'OTM Entry' : otm_entry,
                                "Exit Price": exit,
                                "Exit date": pe_df.iloc[i]["datetime"].date(),
                                "Exit Time": exit_time,
                                'RSI on Entry': entry_rsi,
                                # 'OTM EXIT ' : otm_exit,
                                "Remark": remark,
                                "Points Captured": points_captured,
                                "Slippage": slippage,
                                # 'OTM cost' : otm_exit-otm_entry,
                                "Qty": qty,
                                "PnL": pnl,
                                "ROI%": (pnl / PORTFOLIO_VALUE) * 100,
                                "Trade Year": pe_df.iloc[i]["datetime"].year,
                                "Trade Month": pe_df.iloc[i]["datetime"].month,
                                "Highest High": pe_highest_high,  # Add highest high to trade data
                                "Lowest Low": pe_lowest_low,  # Add lowest low to trade data
                                "Max ROI%": (
                                    (qty * (entry - pe_lowest_low)) / PORTFOLIO_VALUE
                                )
                                * 100,
                                "Margin": (
                                    (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                                )
                                * 100,
                            }
                            # print(trade)
                            trade_book.append(trade)
                            tsl_high = 0
                            points_captured = 0
                            current_date = pe_df.iloc[i]["datetime"].date()
                            current_date_increament_flag = True
                            time_of_day = pe_df.iloc[i]["datetime"].time()
                            # print(f'current date changed to : {current_date} and time to {time_of_day}')
                            break

                        if (
                            in_pe_trade
                            and pe_df.iloc[i]["datetime"].date() == nearest_expiry
                            and pe_df.iloc[i]['datetime'].time() >= dt.time(15, 20)
                        ):
                            # print(pe_df.iloc[i])
                            # print(f'EOD exit {current_candle_close}')
                            # print(f'EOD datetime {pe_df.iloc[i]["datetime"]}')
                            exit = current_candle_close
                            otm_datetime = pe_df.iloc[i]["datetime"]
                            in_pe_trade = False
                            previous_pe_sl_hit = True
                            is_gap_pe_sl = False
                            points_captured = entry - exit
                            exit_time = pe_df.iloc[i]["datetime"].time()
                            slippage = SLIPPAGE * (entry + exit)
                            pnl = qty * (points_captured - slippage)
                            # pnl=(qty*(points_captured-slippage))-qty*(otm_exit-otm_entry)
                            remark = "EOD exit"
                            weekday_int = entry_date.weekday()
                            weekday_name = [
                                "Monday",
                                "Tuesday",
                                "Wednesday",
                                "Thursday",
                                "Friday",
                                "Saturday",
                                "Sunday",
                            ][weekday_int]
                            trade = {
                                "date": entry_date,
                                "day": weekday_name,
                                "expiry": expiry,
                                "DTE": (nearest_expiry - entry_date).days,
                                # 'atm' : atm,
                                # 'scrip' : index ,
                                "strike": strike,
                                "type": asset_class,
                                "Entry Price": entry,
                                "Entry Time": entry_time,
                                "initial sl": initial_sl,
                                # "TSL": tsl_high,
                                # 'OTM Entry' : otm_entry,
                                "Exit Price": exit,
                                "Exit date": pe_df.iloc[i]["datetime"].date(),
                                "Exit Time": exit_time,
                                'RSI on Entry': entry_rsi,
                                # 'OTM EXIT ' : otm_exit,
                                "Remark": remark,
                                "Points Captured": points_captured,
                                "Slippage": slippage,
                                # 'OTM cost' : otm_exit-otm_entry,
                                "Qty": qty,
                                "PnL": pnl,
                                "ROI%": (pnl / PORTFOLIO_VALUE) * 100,
                                "Trade Year": pe_df.iloc[i]["datetime"].year,
                                "Trade Month": pe_df.iloc[i]["datetime"].month,
                                "Highest High": pe_highest_high,  # Add highest high to trade data
                                "Lowest Low": pe_lowest_low,  # Add lowest low to trade data
                                "Max ROI%": (
                                    (qty * (entry - pe_lowest_low)) / PORTFOLIO_VALUE
                                )
                                * 100,
                                "Margin": (
                                    (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                                )
                                * 100,
                            }
                            # print(trade)
                            trade_book.append(trade)
                            # tsl_high = 0
                            points_captured = 0
                            current_date = nearest_expiry + dt.timedelta(days=1)
                            current_date_increament_flag = True
                            time_of_day = dt.time(9, 15)
                            # print(f'current date increased by 1 on expiry : {current_date}')
                            break

                        if (
                            not in_pe_trade
                            and not previous_pe_sl_hit
                            and pe_df.iloc[i]["datetime"].time() > dt.time(15, 15)
                        ):
                            # print('inside exoiry non trade date increment')
                            current_date = current_date + dt.timedelta(days=1)
                            current_date_increament_flag = True
                            time_of_day = dt.time(9, 15)
                            break

        if not current_date_increament_flag:
            current_date = current_date + dt.timedelta(days=1)
            current_date_increament_flag = False

    trade_book_df = pd.DataFrame(trade_book)

    return trade_book_df

In [18]:
async def execute(DF, n, rsi_n, rsi_overbought):
    data = DF.copy()
    tb_ce = await ce_trade(data, n, rsi_n, rsi_overbought)
    tb_pe = await pe_trade(data, n, rsi_n, rsi_overbought)
    tb = pd.concat([tb_ce, tb_pe], ignore_index=True)
    # print(len(tb))
    if len(tb)>0:
        tb = tb.sort_values(by="date")
    return tb

In [19]:
def generate_stats(tb_expiry, ema_window):
    stats_df8 = pd.DataFrame(
        index=range(2019, 2026),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe

    # Iterate over each year
    for year in range(2019, 2026):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()

        # Calculate total number of trades
        total_trades = len(year_trades)

        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100

        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()

        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()

        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()

        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = f"{ema_window}"

        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]

    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0][
        "ROI%"
    ].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum()
        - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation

    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio: stats_df8}

In [24]:
# short_ma = 4
# long_ma = 12
# signal_window = 9
# ema_window = 25
# n = 4
# rsi_n = 6
# rsi_overbought = 45

tb = await execute(data, 49, 50000, 0.55)

2019-01-01
2019-01-02
2019-01-03
2019-01-04
2019-01-05
2019-01-06
2019-01-07
2019-01-08
2019-01-09
2019-01-10
2019-01-11
2019-01-12
2019-01-13
2019-01-14
2019-01-15
2019-01-16
2019-01-17
2019-01-18
2019-01-19
2019-01-20
2019-01-21
2019-01-22
2019-01-23
2019-01-24
2019-01-25
2019-02-01
2019-02-02
2019-02-03
2019-02-04
2019-02-05
2019-02-06
2019-02-07
2019-02-08
2019-02-09
2019-02-10
2019-02-11
2019-02-12
2019-02-15
2019-02-16
2019-02-17
2019-02-18
2019-02-22
2019-02-23
2019-02-24
2019-02-25
2019-02-26
2019-02-26
2019-02-27
2019-02-28
2019-03-01
2019-03-02
2019-03-03
2019-03-04
2019-03-05
2019-03-05
2019-03-06
2019-03-07
2019-03-08
2019-03-09
2019-03-10
2019-03-11
2019-03-12
2019-03-13
2019-03-14
2019-03-15
2019-03-16
2019-03-17
2019-03-18
2019-03-19
2019-03-20
2019-03-21
2019-03-22
2019-03-23
2019-03-24
2019-03-25
2019-03-26
2019-03-27
2019-03-28
2019-03-29
2019-03-30
2019-03-31
2019-04-01
2019-04-02
2019-04-02
2019-04-03
2019-04-05
2019-04-06
2019-04-07
2019-04-08
2019-04-09
2019-04-10

In [25]:
tb['DATETIME'] = pd.to_datetime(tb['date'].astype(str) + ' ' + tb['Entry Time'].astype(str))

In [26]:
tb = tb.sort_values(by='DATETIME')
stats = generate_stats(tb, '...')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,81.3374,86,58.1395,3.6377,-2.7929,-12.1393,6.7004,...
2020,120.1328,109,57.7982,4.1602,-3.0861,-12.5725,9.5552,...
2021,88.7812,102,52.9412,4.4061,-3.1072,-19.6211,4.5248,...
2022,81.4753,105,54.2857,4.0705,-3.1363,-14.5782,5.5888,...
2023,56.6479,100,52.0000,3.6718,-2.7977,-19.0605,2.9720,...
2024,116.2950,95,60.0000,3.9316,-2.8370,-12.0344,9.6635,...
2025,31.5952,18,77.7778,3.1414,-3.0960,-3.4687,9.1086,...
Overall,576.2647,615,56.4228,3.9566,-2.9727,-20.4286,28.2087,...


In [27]:
tb.tail(20)

,date,day,expiry,DTE,strike,type,Entry Price,Entry Time,initial sl,Exit Price,Exit date,Exit Time,RSI on Entry,Remark,Points Captured,Slippage,Qty,PnL,ROI%,Trade Year,Trade Month,Highest High,Lowest Low,Max ROI%,Margin,DATETIME
288,2024-12-20,Friday,2024-12-26,6,24000,C,141.1000,12:05:00,266.9500,0.0500,2024-12-26,15:25:00,0,EOD exit,141.0500,1.4115,238.3790,33286.8892,3.3287,2024,12,143.0000,0.0500,3.3623,71.5137,2024-12-20 12:05:00
605,2024-12-26,Thursday,2024-12-26,0,23750,P,22.2000,09:25:00,50.3500,50.3500,2024-12-26,09:55:00,0,SL hit,-28.1500,0.7255,842.1053,-24316.2105,-2.4316,2024,12,59.0000,17.6000,0.3874,250.0000,2024-12-26 09:25:00
289,2025-01-06,Monday,2025-01-09,3,24050,C,116.0000,09:25:00,156.1000,0.0500,2025-01-09,15:25:00,0,EOD exit,115.9500,1.1605,748.1297,85877.4314,8.5877,2025,1,152.7500,0.0500,8.6746,224.9065,2025-01-06 09:25:00
290,2025-01-13,Monday,2025-01-16,3,23200,C,138.5000,12:55:00,260.0500,111.4000,2025-01-16,15:25:00,0,EOD exit,27.1000,2.4990,246.8120,6071.8223,0.6072,2025,1,162.7000,43.4000,2.3472,71.5755,2025-01-13 12:55:00
606,2025-01-14,Tuesday,2025-01-16,2,23150,P,55.2500,10:55:00,108.5000,0.0500,2025-01-16,15:25:00,0,EOD exit,55.2000,0.5530,563.3803,30787.0423,3.0787,2025,1,82.9500,0.0500,3.1099,163.0282,2025-01-14 10:55:00
607,2025-01-20,Monday,2025-01-23,3,23250,P,107.4500,11:15:00,213.2500,213.2500,2025-01-21,10:15:00,0,SL hit,-105.8000,3.2070,283.5539,-30909.3573,-3.0909,2025,1,221.9000,57.2000,1.4249,82.4078,2025-01-20 11:15:00
291,2025-01-21,Tuesday,2025-01-23,2,23450,C,61.6500,10:15:00,125.0500,0.0500,2025-01-23,15:25:00,0,EOD exit,61.6000,0.6170,473.1861,28856.3091,2.8856,2025,1,123.4000,0.0500,2.9148,138.7027,2025-01-21 10:15:00
608,2025-01-22,Wednesday,2025-01-23,1,23050,P,39.8500,14:15:00,106.3500,0.0500,2025-01-23,15:25:00,0,EOD exit,39.8000,0.3990,451.1278,17774.8872,1.7775,2025,1,48.4500,0.0500,1.7955,129.9812,2025-01-22 14:15:00
609,2025-01-24,Friday,2025-01-30,6,23150,P,104.8000,11:05:00,212.4500,212.4500,2025-01-24,14:35:00,0,SL hit,-107.6500,3.1725,278.6809,-30884.1152,-3.0884,2025,1,220.0000,86.0500,0.5225,80.6433,2025-01-24 11:05:00
292,2025-01-24,Friday,2025-01-30,6,23200,C,149.7500,14:05:00,280.0000,49.7500,2025-01-30,15:25:00,0,EOD exit,100.0000,1.9950,230.3263,22573.1286,2.2573,2025,1,164.4500,6.3500,3.3029,66.7946,2025-01-24 14:05:00


In [29]:
tb = tb.sort_values(by='DATETIME')
stats = generate_stats(tb, '...')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,81.3374,86,58.1395,3.6377,-2.7929,-12.1393,6.7004,...
2020,120.1328,109,57.7982,4.1602,-3.0861,-12.5725,9.5552,...
2021,88.7812,102,52.9412,4.4061,-3.1072,-19.6211,4.5248,...
2022,81.4753,105,54.2857,4.0705,-3.1363,-14.5782,5.5888,...
2023,56.6479,100,52.0000,3.6718,-2.7977,-19.0605,2.9720,...
2024,100.8016,92,58.6957,3.8636,-2.8377,-12.0344,8.3761,...
2025,6.4756,14,57.1429,2.9326,-2.8309,-6.2171,1.0416,...
Overall,535.6517,608,55.5921,3.9555,-2.9678,-20.4286,26.2206,...


In [30]:
26.45

26.45

In [31]:
tb_margin = tb[tb['Margin'] > 100]
tb_margin['ROI%'].sum()

383.22795568450874

In [53]:
stats = generate_stats(tb, '...')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,27.4080,86,58.1395,1.2673,-0.9989,-3.7083,7.3910,...
2020,40.9333,109,57.7982,1.4129,-1.0452,-4.1912,9.7666,...
2021,27.8955,102,52.9412,1.4697,-1.0723,-6.4768,4.3070,...
2022,26.3496,105,54.2857,1.3649,-1.0719,-4.8874,5.3913,...
2023,22.6600,100,52.0000,1.4350,-1.0825,-6.6213,3.4223,...
2024,37.6179,90,60.0000,1.3774,-1.0212,-4.1502,9.0641,...
Overall,182.8643,592,55.7432,1.3895,-1.0522,-6.8614,26.6512,...


# Variation : Close below 49 candles LOW, Current High > 0.55 * Day High

# 10min BT Running for candle close below prev n candles low variation

In [32]:
tb['Cumulative ROI%'] = tb['ROI%'].cumsum()
tb['Max Cumulative ROI%'] = tb['Cumulative ROI%'].cummax()  # Maximum value so far
tb['DD'] = tb['Cumulative ROI%'] - tb['Max Cumulative ROI%']  # Drawdown
tb

,date,day,expiry,DTE,strike,type,Entry Price,Entry Time,initial sl,Exit Price,Exit date,Exit Time,RSI on Entry,Remark,Points Captured,Slippage,Qty,PnL,ROI%,Trade Year,Trade Month,Highest High,Lowest Low,Max ROI%,Margin,DATETIME,Cumulative ROI%,Max Cumulative ROI%,DD
0,2019-01-25,Friday,2019-01-31,6,10900,C,49.8500,14:05:00,108.9500,0.0500,2019-01-31,15:25:00,0,EOD exit,49.8000,0.4990,507.6142,25025.8883,2.5026,2019,1,52.9000,0.0500,2.5279,69.1624,2019-01-25 14:05:00,2.5026,2.5026,0.0000
294,2019-02-12,Tuesday,2019-02-14,2,10850,P,24.1500,11:25:00,38.6000,38.6000,2019-02-12,14:35:00,0,SL hit,-14.4500,0.6275,1843.3180,-27792.6267,-2.7793,2019,2,44.0000,20.3500,0.7005,250.0000,2019-02-12 11:25:00,-0.2767,2.5026,-2.7793
1,2019-02-12,Tuesday,2019-02-14,2,10900,C,34.0000,14:35:00,62.1000,0.0500,2019-02-14,15:25:00,0,EOD exit,33.9500,0.3405,1067.6157,35882.0285,3.5882,2019,2,39.7000,0.0500,3.6246,145.4626,2019-02-12 14:35:00,3.3115,3.3115,0.0000
2,2019-02-18,Monday,2019-02-21,3,10750,C,41.0500,11:05:00,72.9000,35.0000,2019-02-21,15:25:00,0,EOD exit,6.0500,0.7605,941.9152,4982.2606,0.4982,2019,2,56.7500,5.8000,3.3203,126.5699,2019-02-18 11:05:00,3.8098,3.8098,0.0000
295,2019-02-25,Monday,2019-02-28,3,10800,P,46.4000,09:15:00,53.9500,53.9500,2019-02-25,09:25:00,0,SL hit,-7.5500,1.0035,1851.8519,-15839.8148,-1.5840,2019,2,55.3000,46.4000,0.0000,250.0000,2019-02-25 09:15:00,2.2258,3.8098,-1.5840
296,2019-02-25,Monday,2019-02-28,3,10800,P,42.3500,12:05:00,60.6500,60.6500,2019-02-26,09:15:00,0,SL hit,-18.3000,1.0300,1639.3443,-31688.5246,-3.1689,2019,2,76.8500,22.3500,3.2787,221.3115,2019-02-25 12:05:00,-0.9431,3.8098,-4.7528
3,2019-02-26,Tuesday,2019-02-28,2,10800,C,58.7000,09:35:00,86.2500,86.2500,2019-02-26,11:45:00,0,SL hit,-27.5500,1.4495,1088.9292,-31578.4029,-3.1578,2019,2,89.1500,42.2000,1.7967,147.0054,2019-02-26 09:35:00,-4.1009,3.8098,-7.9107
297,2019-03-01,Friday,2019-03-07,6,10800,P,48.3000,14:25:00,68.6500,0.0500,2019-03-07,15:25:00,0,EOD exit,48.2500,0.4835,1474.2015,70417.4447,7.0417,2019,3,55.0000,0.0500,7.1130,199.0172,2019-03-01 14:25:00,2.9408,3.8098,-0.8689
4,2019-03-05,Tuesday,2019-03-07,2,10900,C,33.9000,09:15:00,52.8000,52.8000,2019-03-05,11:55:00,0,SL hit,-18.9000,0.8670,1587.3016,-31376.1905,-3.1376,2019,3,54.6500,29.0000,0.7778,216.2698,2019-03-05 09:15:00,-0.1968,3.8098,-4.0065
298,2019-03-11,Monday,2019-03-14,3,11050,P,37.1500,09:15:00,67.1500,0.0500,2019-03-14,15:25:00,0,EOD exit,37.1000,0.3720,1000.0000,36728.0000,3.6728,2019,3,38.3500,0.0500,3.7100,138.1250,2019-03-11 09:15:00,3.4760,3.8098,-0.3337


In [33]:
tb.to_csv('NLC_New_10m.csv')

In [24]:
#10min
stats_dictionary = {}
for i in range(12, 49, 4):
    for j in range(20, 71, 10):
        print(f'{i}, {j/100}%')
        tb = await execute(data, i, 5, j/100)
        if len(tb) > 0:
            stats = generate_stats(tb, f'{i}, {j/100}%')
            for x, y in stats.items():
                z = pd.DataFrame(y)
                print(z.to_string())
                stats_dictionary[x] = y

12, 0.2%


/tmp/ipykernel_59697/1336650796.py:146: RuntimeWarning: divide by zero encountered in scalar divide
  qty = RPT_CE * PORTFOLIO_VALUE / (initial_sl - entry)


        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio Variation
2019     173.4280          172  51.1628                4.7549             -2.9168     -19.2316       9.0179  12, 0.2%
2020     188.7499          210  45.2381                5.5258             -2.9235     -15.2785      12.3540  12, 0.2%
2021     101.4692          199  45.2261                4.6568             -2.9142     -30.0203       3.3800  12, 0.2%
2022      46.0617          218  42.2018                5.2306             -3.4536     -51.5416       0.8937  12, 0.2%
2023      47.6809          199  44.2211                4.0742             -2.8004     -26.7550       1.7821  12, 0.2%
2024      41.7527          150  40.6667                4.7623             -2.7949     -39.1397       1.0668  12, 0.2%
Overall  599.1424         1148  44.7735                4.8497             -2.9867     -51.5416      11.6244  12, 0.2%
12, 0.3%


/tmp/ipykernel_59697/1336650796.py:146: RuntimeWarning: divide by zero encountered in scalar divide
  qty = RPT_CE * PORTFOLIO_VALUE / (initial_sl - entry)


        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio Variation
2019     172.8051          170  50.5882                4.8583             -2.9168     -22.4432       7.6997  12, 0.3%
2020     187.6521          209  44.9761                5.5729             -2.9235     -15.2785      12.2821  12, 0.3%
2021     100.9231          198  44.9495                4.7030             -2.9142     -30.0203       3.3618  12, 0.3%
2022      46.0617          218  42.2018                5.2306             -3.4536     -55.3935       0.8315  12, 0.3%
2023      45.8014          195  43.0769                4.2458             -2.8004     -32.6027       1.4048  12, 0.3%
2024      40.5747          148  39.8649                4.9037             -2.7949     -37.8598       1.0717  12, 0.3%
Overall  593.8180         1138  44.2882                4.9353             -2.9867     -55.3935      10.7200  12, 0.3%
12, 0.4%


...

        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio Variation
2019     137.4240          141  56.0284                3.9063             -2.7608     -15.8885       8.6493  24, 0.4%
2020     199.4769          172  50.5814                5.0862             -2.8590     -13.6529      14.6106  24, 0.4%
2021      78.3394          169  49.7041                4.0433             -3.0741     -31.1025       2.5187  24, 0.4%
2022      32.6532          178  43.2584                4.6343             -3.2098     -37.0854       0.8805  24, 0.4%
2023      59.5495          163  46.0123                3.9260             -2.6693     -26.8461       2.2182  24, 0.4%
2024      60.4861          118  42.3729                4.8303             -2.6622     -21.7193       2.7849  24, 0.4%
Overall  567.9291          941  48.0340                4.3884             -2.8949     -37.0854      15.3141  24, 0.4%
24, 0.5%


/tmp/ipykernel_59697/1336650796.py:146: RuntimeWarning: divide by zero encountered in scalar divide
  qty = RPT_CE * PORTFOLIO_VALUE / (initial_sl - entry)


        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio Variation
2019     136.2880          128  55.4688                4.2017             -2.8427     -18.5782       7.3359  24, 0.5%
2020     183.2379          159  50.9434                5.0954             -2.9421     -12.5276      14.6267  24, 0.5%
2021      81.1690          155  47.7419                4.5487             -3.1536     -29.3922       2.7616  24, 0.5%
2022      43.8370          170  44.1176                4.6993             -3.2485     -32.5623       1.3463  24, 0.5%
2023      46.6809          157  45.2229                3.9892             -2.7506     -29.1675       1.6004  24, 0.5%
2024      69.7482          110  43.6364                5.0277             -2.7674     -13.8796       5.0252  24, 0.5%
Overall  560.9610          879  47.7816                4.5825             -2.9710     -32.5623      17.2273  24, 0.5%
24, 0.6%


/tmp/ipykernel_59697/1336650796.py:146: RuntimeWarning: divide by zero encountered in scalar divide
  qty = RPT_CE * PORTFOLIO_VALUE / (initial_sl - entry)


        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio Variation
2019     109.9116          115  50.4348                4.6642             -2.8178     -24.4122       4.5023  24, 0.6%
2020     155.0865          147  51.7007                4.8933             -3.0536     -12.5276      12.3796  24, 0.6%
2021      59.0946          141  44.6809                4.9351             -3.2284     -33.6146       1.7580  24, 0.6%
2022      47.6104          151  43.7086                4.9703             -3.2992     -24.2691       1.9618  24, 0.6%
2023      34.9851          138  44.2029                4.1897             -2.8648     -30.5597       1.1448  24, 0.6%
2024      61.5091           97  42.2680                5.4380             -2.8830     -14.9063       4.1264  24, 0.6%
Overall  468.1974          789  46.2611                4.8216             -3.0465     -33.6146      13.9284  24, 0.6%
24, 0.7%
        Total ROI Total Trades Win Rate Avg Pro

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_sum_roi_by_dte(tb):
    """
    Plots the sum of ROI% across all unique DTE values.

    Parameters:
    tb (pd.DataFrame): Input DataFrame with 'DTE' and 'ROI%' columns.
    """
    # Ensure the DataFrame contains the required columns
    required_columns = {'DTE', 'ROI%'}
    if not required_columns.issubset(tb.columns):
        raise ValueError(f"Input DataFrame must contain columns: {required_columns}")
    
    # Group by DTE and calculate the sum of ROI%
    result = tb.groupby('DTE', as_index=False)['ROI%'].sum()
    result.rename(columns={'ROI%': 'Sum_ROI%'}, inplace=True)

    # Plot the data
    plt.figure(figsize=(10, 6))
    plt.bar(result['DTE'], result['Sum_ROI%'], color='skyblue', edgecolor='black')
    
    # Add labels and title
    plt.title('Sum of ROI% Across Unique DTE Values', fontsize=14)
    plt.xlabel('DTE', fontsize=12)
    plt.ylabel('Sum of ROI%', fontsize=12)
    plt.xticks(rotation=45, fontsize=10)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Show the plot
    plt.tight_layout()
    plt.show()

# Example usage
# tb = pd.DataFrame({'DTE': [...], 'ROI%': [...]})  # Replace with your actual DataFrame
plot_sum_roi_by_dte(tb)


In [92]:
def calculate_max_dd(tb):
    """
    Calculates the maximum drawdown for each group of 'DTE'.
    
    Parameters:
    tb (pd.DataFrame): Input DataFrame with 'DTE' and 'ROI%' columns.
    
    Returns:
    pd.Series: Maximum drawdown for each unique DTE.
    """
    # Calculate cumulative sum of ROI%
    cumulative_returns = tb["ROI%"].cumsum()
    
    # Calculate maximum drawdown
    max_drawdown = (cumulative_returns - cumulative_returns.cummax()).min()
    
    return max_drawdown


In [ ]:
def plot_sum_roi_and_max_dd_by_dte(tb):
    """
    Plots the sum of ROI% and Absolute Max Drawdown (Max DD) across all unique DTE values
    using a single Y-axis, and prints the resulting DataFrame.

    Parameters:
    tb (pd.DataFrame): Input DataFrame with 'DTE' and 'ROI%' columns.
    """
    # Ensure the DataFrame contains the required columns
    required_columns = {'DTE', 'ROI%'}
    if not required_columns.issubset(tb.columns):
        raise ValueError(f"Input DataFrame must contain columns: {required_columns}")
    
    # Group by DTE and calculate the sum of ROI%
    roi_result = tb.groupby('DTE', as_index=False)['ROI%'].sum()
    roi_result.rename(columns={'ROI%': 'Sum_ROI%'}, inplace=True)

    # Group by DTE and calculate Max DD (in absolute terms)
    dd_result = tb.groupby('DTE').apply(calculate_max_dd).reset_index()
    dd_result.rename(columns={0: 'Max_DD%'}, inplace=True)
    dd_result['Max_DD%'] = dd_result['Max_DD%'].abs()

    # Merge the sum of ROI% and Max DD data
    result = pd.merge(roi_result, dd_result, on='DTE')

    # Add ROI% / Max DD ratio
    result['ROI/DD_Ratio'] = result.apply(
        lambda row: row['Sum_ROI%'] / row['Max_DD%'] if row['Max_DD%'] != 0 else float('inf'), axis=1
    )

    # Format values
    result['Sum_ROI%'] = result['Sum_ROI%'].round(2)
    result['Max_DD%'] = result['Max_DD%'].round(2)
    result['ROI/DD_Ratio'] = result['ROI/DD_Ratio'].round(2)

    # Print the DataFrame
    print("\nResulting DataFrame:")
    print(result)

    # Plot the data
    fig, ax = plt.subplots(figsize=(10, 6))

    # Plot Sum of ROI%
    ax.bar(result['DTE'], result['Sum_ROI%'], color='skyblue', edgecolor='black', label='Sum of ROI%', width=0.4, align='center')
    
    # Overlay Max DD (absolute values) on the same axis
    ax.plot(result['DTE'], result['Max_DD%'], color='red', marker='o', label='Max Drawdown (absolute)', linestyle='--', linewidth=2)

    ax.set_xlabel('DTE', fontsize=12)
    ax.set_ylabel('Values', fontsize=12)
    ax.tick_params(axis='x', rotation=45)

    # Title, legend, and grid
    plt.title('Sum of ROI%, Absolute Max Drawdown, and ROI/DD Ratio', fontsize=14)
    ax.legend(loc='upper left', fontsize=10)
    ax.grid(axis='y', linestyle='--', alpha=0.7)

    # Show the plot
    fig.tight_layout()
    plt.show()

plot_sum_roi_and_max_dd_by_dte(tb)


In [ ]:
def plot_sum_roi_and_max_dd_by_dte(tb):
    """
    Plots the sum of ROI% and Absolute Max Drawdown (Max DD) across all unique DTE values
    using a single Y-axis, and prints the resulting DataFrame.

    Parameters:
    tb (pd.DataFrame): Input DataFrame with 'DTE' and 'ROI%' columns.
    """
    # Ensure the DataFrame contains the required columns
    required_columns = {'DTE', 'ROI%'}
    if not required_columns.issubset(tb.columns):
        raise ValueError(f"Input DataFrame must contain columns: {required_columns}")
    
    # Group by DTE and calculate the sum of ROI%
    roi_result = tb.groupby('DTE', as_index=False)['ROI%'].sum()
    roi_result.rename(columns={'ROI%': 'Sum_ROI%'}, inplace=True)

    # Group by DTE and calculate Max DD (in absolute terms)
    dd_result = tb.groupby('DTE').apply(calculate_max_dd).reset_index()
    dd_result.rename(columns={0: 'Max_DD%'}, inplace=True)
    dd_result['Max_DD%'] = dd_result['Max_DD%'].abs()

    # Merge the sum of ROI% and Max DD data
    result = pd.merge(roi_result, dd_result, on='DTE')

    # Add ROI% / Max DD ratio
    result['ROI/DD_Ratio'] = result.apply(
        lambda row: row['Sum_ROI%'] / row['Max_DD%'] if row['Max_DD%'] != 0 else float('inf'), axis=1
    )

    # Format values
    result['Sum_ROI%'] = result['Sum_ROI%'].round(2)
    result['Max_DD%'] = result['Max_DD%'].round(2)
    result['ROI/DD_Ratio'] = result['ROI/DD_Ratio'].round(2)

    # Print the DataFrame
    print("\nResulting DataFrame:")
    print(result)

    # Plot the data
    fig, ax = plt.subplots(figsize=(10, 6))

    # Plot Sum of ROI%
    ax.bar(result['DTE'], result['Sum_ROI%'], color='skyblue', edgecolor='black', label='Sum of ROI%', width=0.4, align='center')
    
    # Overlay Max DD (absolute values) on the same axis
    ax.plot(result['DTE'], result['Max_DD%'], color='red', marker='o', label='Max Drawdown (absolute)', linestyle='--', linewidth=2)

    ax.set_xlabel('DTE', fontsize=12)
    ax.set_ylabel('Values', fontsize=12)
    ax.tick_params(axis='x', rotation=45)

    # Title, legend, and grid
    plt.title('Sum of ROI%, Absolute Max Drawdown, and ROI/DD Ratio', fontsize=14)
    ax.legend(loc='upper left', fontsize=10)
    ax.grid(axis='y', linestyle='--', alpha=0.7)

    # Show the plot
    fig.tight_layout()
    plt.show()

plot_sum_roi_and_max_dd_by_dte(tb)


In [25]:
# 3min TF
sorted_stats = {k: v for k, v in sorted(stats_dictionary.items(), key=lambda item: item[0], reverse=True)}
for x, y in sorted_stats.items():
    print(y.to_string())

        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio Variation
2019      85.9035           97  59.7938                3.3811             -2.8256     -12.1311       7.0813  48, 0.5%
2020     127.9911          118  57.6271                4.1027             -3.0198     -14.5938       8.7702  48, 0.5%
2021      89.5494          110  53.6364                4.2052             -3.1090     -20.1005       4.4551  48, 0.5%
2022     106.5603          106  59.4340                3.7684             -3.0430     -12.0577       8.8375  48, 0.5%
2023      44.0887          114  50.8772                3.4633             -2.7997     -20.6555       2.1345  48, 0.5%
2024      57.8441           75  56.0000                3.6423             -2.8828     -10.0489       5.7563  48, 0.5%
Overall  511.9372          620  56.1290                3.7772             -2.9504     -20.6555      24.7846  48, 0.5%
        Total ROI Total Trades Win Rate Avg Profit% per 